In [13]:
import tensorflow as tf
# from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

In [14]:
# Check for GPU availability
print("Available GPUs:", tf.config.list_physical_devices('GPU'))

Available GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [15]:
# Set batch size
batch_size = 16

In [16]:
# Data augmentation and preparation
train_data_generator = ImageDataGenerator(rescale=1./255, rotation_range=0.2, shear_range=0.2, zoom_range=0.2, 
                                          width_shift_range=0.2, height_shift_range=0.2, validation_split=0.2,horizontal_flip=True)

In [17]:
# Train and validation data
train_data = train_data_generator.flow_from_directory(
    'data/train',
    target_size=(80, 80),  # Updated size for EfficientNetB0
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_data = train_data_generator.flow_from_directory(
    'data/train',
    target_size=(80, 80),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


Found 64524 images belonging to 2 classes.
Found 16130 images belonging to 2 classes.


In [18]:
# Test data preparation
test_data_generator = ImageDataGenerator(rescale=1./255)
test_data = test_data_generator.flow_from_directory(
    'data/test',
    target_size=(80, 80),
    batch_size=batch_size,
    class_mode='categorical'
)


Found 4244 images belonging to 2 classes.


In [19]:
# Compute class weights to handle class imbalance
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_data.classes),
    y=train_data.classes
)
class_weights = dict(enumerate(class_weights))

In [20]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(80, 80, 3))
# Access the output of the base model
head_model = base_model.output
# Adding a fully connected layer of finetuning
head_model = Flatten()(head_model)
head_model = Dense(128, activation='relu')(head_model)
head_model = Dropout(0.5)(head_model)
head_model = Dense(64, activation='relu')(head_model)
head_model = Dropout(0.5)(head_model)
head_model = Dense(2, activation='softmax')(head_model)

model = Model(inputs=base_model.input, outputs=head_model)
for layer in base_model.layers:
    layer.trainable = False

In [21]:
# Save the best model as best_model.h5
checkpoint = ModelCheckpoint(
    filepath='save_models/InceptionV3_best_model.h5',  # Save the best model as best_model.h5
    monitor='val_loss',       # Monitor validation loss
    verbose=3,                # Verbose output to show progress
    save_best_only=True,      # Save only the best model (not every epoch)
    save_weights_only=False   # Save the entire model, not just the weights
)
earlystop = EarlyStopping(monitor='val_loss', 
                          patience=7, 
                          verbose=3, 
                          restore_best_weights=True)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=3, 
                                            verbose=3)

callbacks = [checkpoint, earlystop, learning_rate_reduction]

In [22]:
callbacks = [checkpoint, earlystop, learning_rate_reduction]

In [23]:
# Recompile the model after unfreezing
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),  # Smaller learning rate for fine-tuning
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [24]:
# Train the model
history = model.fit(
    train_data,
    steps_per_epoch=train_data.samples // batch_size,
    validation_data=val_data,
    validation_steps=val_data.samples // batch_size,
    callbacks=callbacks,
    epochs=30,
    class_weight=class_weights  # Handle class imbalance
)


Epoch 1/30
4031/4032 [============================>.] - ETA: 0s - loss: 0.4750 - accuracy: 0.7723
Epoch 1: val_loss improved from inf to 0.32587, saving model to save_models\InceptionV3_best_model.h5
4032/4032 [==============================] - 156s 38ms/step - loss: 0.4749 - accuracy: 0.7724 - val_loss: 0.3259 - val_accuracy: 0.8493 - lr: 1.0000e-05
Epoch 2/30
4030/4032 [============================>.] - ETA: 0s - loss: 0.3070 - accuracy: 0.8765
Epoch 2: val_loss improved from 0.32587 to 0.29441, saving model to save_models\InceptionV3_best_model.h5
4032/4032 [==============================] - 126s 31ms/step - loss: 0.3070 - accuracy: 0.8765 - val_loss: 0.2944 - val_accuracy: 0.8669 - lr: 1.0000e-05
Epoch 3/30
4031/4032 [============================>.] - ETA: 0s - loss: 0.2628 - accuracy: 0.8983
Epoch 3: val_loss improved from 0.29441 to 0.27681, saving model to save_models\InceptionV3_best_model.h5
4032/4032 [==============================] - 182s 45ms/step - loss: 0.2628 - accuracy: